In [1]:
import ipywidgets as widgets

In [2]:
import numpy as np

def generate_data():
    n = 50
    a = 1.5
    b = 0.3

    X = np.random.random(n)
    y = a * X + b + np.random.random(n) / 2

    return X, y

In [3]:
X, y = generate_data()

In [4]:
from optimization import GurobiSolver
from kernel import LinearKernel, GaussianKernel
import matplotlib.pyplot as plt
s = GurobiSolver()


In [82]:

def feature_dot(x_new, sv, kernel):
    return sum([a * kernel.compute(x, x_new) for (a, x) in sv])


@widgets.interact(C=widgets.FloatSlider(min=1E-3, max=1E3, step=.001, value=1),
                  sigma=widgets.FloatSlider(min=1E-3, max=10, step=0.01, 
                                           value=1),
                  epsilon=widgets.FloatSlider(min=0.01, max=1,
                                              step=.01, value=.1),
                  B=widgets.FloatSlider(min=0.01, max=10, step=.5, value=1)
                  )
def f(C, sigma, epsilon, B):
    k = GaussianKernel(sigma)
    alpha, alpha_hat = s.solve(X, y, C=1, kernel=k, epsilon=epsilon)
    print(k)
    sv = [(a - a_h, x) for a, a_h, x in zip(alpha, alpha_hat, X)
                                     if a - a_h != 0]

    b_values = [y_ - feature_dot(x, sv, k) - epsilon
                for a, x, y_ in zip(alpha, X, y) if 0 < a < C]
    b_values += [y_ - feature_dot(x, sv, k) + epsilon
                for a_h, x, y_ in zip(alpha_hat, X, y) if 0 < a_h < C]

    b = np.mean(b_values)

    regression = lambda x: feature_dot(x, sv, k) + b

    x_values = np.linspace(min(X), max(X), 500)

    fig, ax = plt.subplots()
    ax.scatter(X, y)
    ax.plot(x_values, [regression(x) for x in x_values])
    ax.plot(x_values, [regression(x) + epsilon for x in x_values],
            'k--', linewidth=.5)
    ax.plot(x_values, [regression(x) - epsilon for x in x_values],
            'k--', linewidth=.5)

    alpha, alpha_hat, gamma = s.solve(X, y, C, k, epsilon, budget=B)

    budget_sv = [(a - a_h, x) for a, a_h, x
             in zip(alpha, alpha_hat, X) if a - a_h != 0]


    budget_b_values = [y_ - feature_dot(x, sv, k) - epsilon
                for a, x, y_ in zip(alpha, X, y) if 0 < a < C + gamma]
    budget_b_values += [y_ - feature_dot(x, sv, k) + epsilon
                for a_h, x, y_ in zip(alpha_hat, X, y) if 0 < a_h < C + gamma]

    budget_b = np.mean(b_values)

    budget_regression = lambda x: feature_dot(x, budget_sv, k) + budget_b

    ax.plot(x_values, [budget_regression(x) + epsilon for x in x_values],
            'r:', linewidth=.5)
    ax.plot(x_values, [budget_regression(x) for x in x_values], 'r')
    ax.plot(x_values, [budget_regression(x) - epsilon for x in x_values],
            'r:', linewidth=.5)
    #return alpha, alpha_hat

interactive(children=(FloatSlider(value=1.0, description='C', max=1000.0, min=0.001, step=0.001), FloatSlider(…